In [33]:
import boto3
import spacy
spacy_nlp = spacy.load("en_core_web_sm")
from nltk.tokenize import sent_tokenize
#nltk.download('punkt')
import time

In [5]:
def sentenceSentiment(text):
    """
    Takes in a line of text and uses amazon's comprehend to obtain the sentiment of the text.
    Returns the text and the score, as well as the sentiment of the text
    """
    comprehend = boto3.client(service_name='comprehend', region_name='us-west-2')
    sentiment_json = comprehend.detect_sentiment(Text=text, LanguageCode='en')
    sent = sentiment_json['Sentiment']
    sent_pos = sentiment_json['SentimentScore']['Positive']
    sent_neg = sentiment_json['SentimentScore']['Negative']
    sent_neu = sentiment_json['SentimentScore']['Neutral']
    sent_mix = sentiment_json['SentimentScore']['Mixed']

    return sent, sent_pos, sent_neg, sent_neu, sent_mix
#     if sent == "POSITIVE":
#         return (text,sent_pos), sent
#     elif sent == "NEGATIVE":
#         return (text,sent_neg), sent
#     elif sent == "NEUTRAL":
#         return (text, sent_neu), sent
#     elif sent == "MIXED":
#         return (text, sent_mix), sent

In [12]:
import pandas as pd
df = pd.read_csv("ExampleTexts/videoExamples.csv")
examples = list(df.captions)

In [18]:
examples[5]

' - Alright, it\'s iPhone time. Apple released three new iPhones all at once this year. The iPhone 11, the iPhone 11 Pro, and the iPhone 11 Pro Max. Now, I honestly think the iPhone 11 is the phone most people in the iOS ecosystem should get if they\'re upgrading. So we\'ve got a whole other review of that phone that you should check out, too. But this, this is the Pro review. The dark mode, cash money review for professionals who use their phones to do pro stuff. Like taking telephoto photos. Actually, I have no idea why Apple called these phones Pro. They\'re basically just nice updates to the iPhone XS, but I also think it\'s a waste of time to argue about names. iPhone 11 Pro Max is a bad name. But you know what, it\'s a great phone. And I think Apple might have done it. This is the best camera I\'ve ever seen on a phone. Let\'s get into it. (calming music) The iPhone 11 Pro looks almost exactly like the iPhone XS from the front. It\'s a little heavier and thicker, but unless you\'

In [34]:
curtime = time.time()
for sentence in sent_tokenize(examples[5]):
    print(sentenceSentiment(sentence))
print("{} seconds have passed".format(time.time()-curtime))

('NEUTRAL', 0.22162330150604248, 0.007322193123400211, 0.7710028290748596, 5.167251583770849e-05)
('NEUTRAL', 0.1466148942708969, 0.011058105155825615, 0.8423168063163757, 1.0174547242058907e-05)
('NEUTRAL', 0.007717764005064964, 0.00039099695277400315, 0.9918674826622009, 2.377944838372059e-05)
('NEUTRAL', 0.3294348418712616, 0.12619709968566895, 0.5440768599510193, 0.0002912083291448653)
('NEUTRAL', 0.10908884555101395, 0.01815667934715748, 0.8727174401283264, 3.703484981087968e-05)
('NEUTRAL', 0.008487893268465996, 0.00199640984646976, 0.989510715007782, 5.001821591577027e-06)
('NEUTRAL', 0.06801239401102066, 0.042086947709321976, 0.8898788690567017, 2.175967711082194e-05)
('POSITIVE', 0.8942007422447205, 0.03175944462418556, 0.0740155279636383, 2.4355702407774515e-05)
('NEUTRAL', 0.011034333147108555, 0.3023909628391266, 0.6865245699882507, 5.0053240556735545e-05)
('NEGATIVE', 0.044295985251665115, 0.8905709385871887, 0.04916748031973839, 0.015965662896633148)
('NEGATIVE', 0.026573

('MIXED', 0.03318799287080765, 0.002232879400253296, 0.0031753915827721357, 0.9614038467407227)
('NEGATIVE', 0.002259828383103013, 0.9168183207511902, 0.08090746402740479, 1.432602948625572e-05)
('POSITIVE', 0.9959062337875366, 6.118070450611413e-05, 0.004031946882605553, 6.674392238892324e-07)
('POSITIVE', 0.9531608819961548, 0.003221833147108555, 0.03984018415212631, 0.0037770806811749935)
('POSITIVE', 0.9821723103523254, 0.010722844861447811, 0.007025322876870632, 7.957332127261907e-05)
('NEGATIVE', 0.03418141230940819, 0.5142455101013184, 0.4515494704246521, 2.367753222642932e-05)
('NEUTRAL', 0.29575029015541077, 0.2957265079021454, 0.35472413897514343, 0.05379905179142952)
('POSITIVE', 0.9822593927383423, 0.0003113428538199514, 0.017426952719688416, 2.349619080632692e-06)
('POSITIVE', 0.9130393266677856, 0.0016776274424046278, 0.08528165519237518, 1.4013016880198848e-06)
('POSITIVE', 0.8613272905349731, 0.0029272811952978373, 0.1357424408197403, 2.9442851428029826e-06)
('POSITIVE'

('NEGATIVE', 0.024622038006782532, 0.9369778633117676, 0.038258910179138184, 0.00014119999832473695)
('NEUTRAL', 0.2263539731502533, 0.32934674620628357, 0.4087510406970978, 0.035548195242881775)
('POSITIVE', 0.6599358916282654, 0.19574157893657684, 0.14431560039520264, 6.892939381941687e-06)
('POSITIVE', 0.47566813230514526, 0.3609175384044647, 0.07223708927631378, 0.0911773070693016)
('NEUTRAL', 0.19931015372276306, 0.007499540690332651, 0.7931809425354004, 9.388220860273577e-06)
('NEGATIVE', 0.04533453658223152, 0.48155030608177185, 0.47307920455932617, 3.598035254981369e-05)
('NEUTRAL', 0.08190564066171646, 0.2992602586746216, 0.6185962557792664, 0.00023782052448950708)
('NEGATIVE', 0.06707017868757248, 0.5143677592277527, 0.4183633327484131, 0.00019878878083545715)
('NEGATIVE', 0.029531734064221382, 0.6952959299087524, 0.27516335248947144, 8.895000064512715e-06)
('NEUTRAL', 0.04278567060828209, 0.31734421849250793, 0.6398612856864929, 8.91291620064294e-06)
('POSITIVE', 0.996936082

In [36]:
doc = spacy_nlp(examples[5])

In [51]:
sentences = set()
for token in doc:
    if token.pos_ == "ADJ":
        print(" ".join(word.text for word in token.sent))
        sentences.add(" ".join(word.text for word in token.sent))

Apple released three new iPhones all at once this year .
Now , I honestly think the iPhone 11 is the phone most people in the iOS ecosystem should get if they 're upgrading .
Now , I honestly think the iPhone 11 is the phone most people in the iOS ecosystem should get if they 're upgrading .
So we 've got a whole other review of that phone that you should check out , too .
So we 've got a whole other review of that phone that you should check out , too .
The dark mode , cash money review for professionals who use their phones to do pro stuff .
The dark mode , cash money review for professionals who use their phones to do pro stuff .
They 're basically just nice updates to the iPhone XS , but I also think it 's a waste of time to argue about names .
Pro Max is a bad name .
But you know what , it 's a great phone .
This is the best camera I 've ever seen on a phone .
It 's a little heavier and thicker , but unless you 're comparing them directly , you probably wo n't notice .
It 's a lit

In [46]:
sentences

{", it 's called the U1 , which does precise positioning using an ultra wideband radio .",
 '- But is it really that good ?',
 '11 Pro did a great job of balancing my skin tone and just the exposure in general .',
 '11 Pro has been impressive .',
 '13.1 update will allow you to just point the phone at another U1 device to put it at the top of your AirDrop list .',
 'A lot of people also asked me if this matte finish will scratch easily , similar to the back of the Pixel 3 , and so far the answer seems to be no .',
 'Actually , I have no idea why Apple called these phones Pro .',
 'Add in the improved battery life and the iPhone 11 Pro stands out as a major step forward from the XS and one of the best flagship phones of the year .',
 "Adobe 5.1 sound or game encoded in 7.1 , you 'll get virtual surround from the stereo speakers .",
 'All in all , I think the iPhone 11 camera does better in bright light than the Pixel 3 and Note 10 , and the Night mode beats the Pixel 3 more often than n

In [53]:
curtime = time.time()
for sentence in sentences:
    print(sentenceSentiment(sentence))
print("{} seconds have passed".format(time.time()-curtime))

('POSITIVE', 0.4931589365005493, 0.3073602616786957, 0.1994761824607849, 4.6041714085731655e-06)
('POSITIVE', 0.9862145781517029, 0.010792297311127186, 0.002865136368200183, 0.00012800875992979854)
('NEGATIVE', 0.046247486025094986, 0.9335712194442749, 0.020161723718047142, 1.955099287442863e-05)
('POSITIVE', 0.8357536196708679, 0.014691374264657497, 0.14949342608451843, 6.153246067697182e-05)
('POSITIVE', 0.9931733012199402, 0.0013662326382473111, 0.005440347362309694, 2.0016314010717906e-05)
('POSITIVE', 0.5797889232635498, 0.07619517296552658, 0.34400907158851624, 6.831415703345556e-06)
('NEUTRAL', 0.12548236548900604, 0.025845687836408615, 0.8486600518226624, 1.1888728295161854e-05)
('POSITIVE', 0.9716004729270935, 0.0007389378733932972, 0.027657387778162956, 3.16730893246131e-06)
('NEGATIVE', 0.40748995542526245, 0.5215824246406555, 0.07074683159589767, 0.00018069542420562357)
('POSITIVE', 0.997857391834259, 0.00020344351651147008, 0.0019382110331207514, 9.42491624300601e-07)
('MI

('NEUTRAL', 0.4097396731376648, 0.015563618391752243, 0.5746904611587524, 6.260157533688471e-06)
('NEUTRAL', 0.3293476700782776, 0.011068110354244709, 0.6595779061317444, 6.4025393839983735e-06)
('NEUTRAL', 0.4293133616447449, 0.0003250409208703786, 0.5703403353691101, 2.1312423996278085e-05)
('POSITIVE', 0.997654914855957, 4.838359382119961e-05, 0.002295923884958029, 7.787034519424196e-07)
('POSITIVE', 0.8260369896888733, 0.015589402988553047, 0.1583499163389206, 2.3707239961368032e-05)
('POSITIVE', 0.9991118311882019, 8.34335369290784e-05, 0.0008033591439016163, 1.279584353142127e-06)
('NEUTRAL', 0.3294348418712616, 0.12619709968566895, 0.5440768599510193, 0.0002912083291448653)
('NEUTRAL', 0.19404049217700958, 0.0722593292593956, 0.7336691617965698, 3.108544842689298e-05)
('NEUTRAL', 0.37237268686294556, 0.08533637970685959, 0.5422706604003906, 2.0271983885322697e-05)
('NEGATIVE', 0.33194470405578613, 0.3363940119743347, 0.3316422998905182, 1.907466139527969e-05)
('NEUTRAL', 0.22740

('NEUTRAL', 0.21887458860874176, 0.22232550382614136, 0.5587267279624939, 7.318816642509773e-05)
('NEGATIVE', 0.0058188182301819324, 0.9687682390213013, 0.025401245802640915, 1.171183157566702e-05)
('NEGATIVE', 0.0801151916384697, 0.7691159248352051, 0.1505918651819229, 0.0001770146918715909)
60.6205198764801 seconds have passed


In [52]:
len(sentences)

171

In [50]:
len(sent_tokenize(examples[5]))

224